In [51]:
# json.dump用のクラス
class MyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        elif isinstance(obj, set):
            return list(obj)
        else:
            return super(MyEncoder, self).default(obj)

In [52]:
def count_Matrix(x, mxL):
    """
    負荷率 X がマトリックス mxL の何番目（ix）のセルに入るかをカウント
    """

    # 初期値
    ix = 0

    # C#の処理に合わせる（代表負荷率にする）
    # 負荷率1.00の場合は x=1.05となるため過負荷判定
    x = math.floor(x*10)/10+0.05

    # 該当するマトリックスを探査
    while x > mxL[ix]:
        ix += 1

        if ix == len(mxL)-1:
            break

    return ix+1

def air_enthalpy(Tdb, X):
    """
    空気のエンタルピーを算出する関数
    (WEBPROに合わせる)
    """

    Ca = 1.006  # 乾き空気の定圧比熱 [kJ/kg･K]
    Cw = 1.805  # 水蒸気の定圧比熱 [kJ/kg･K]
    Lw = 2502   # 水の蒸発潜熱 [kJ/kg]

    if len(Tdb) != len(X):
        raise Exception('温度と湿度のリストの長さが異なります。')
    else:

        H = np.zeros(len(Tdb))
        for i in range(0, len(Tdb)):
            H[i] = (Ca*Tdb[i] + (Cw*Tdb[i]+Lw)*X[i])

    return H

In [53]:
file_name = "./sample/WEBPRO_inputSheet_sample.xlsm"
inputfile_name   = str(file_name)
inputfile_name_split = os.path.splitext(inputfile_name)

# 現在の作業ディレクトリを取得
current_directory = os.getcwd()
# sys.pathに追加
sys.path.append(current_directory)

template_directory = os.path.join(current_directory, "builelib/inputdata/")
database_directory = os.path.join(current_directory, "builelib/database/")
climatedata_directory =  os.path.join(current_directory, "builelib/climatedata/")

In [54]:
# テンプレートjsonの読み込み
with open( template_directory + 'template.json', 'r', encoding='utf-8') as f:
  data = json.load(f)

In [55]:
#------------------------------------
# 入力ファイルの読み込み
#------------------------------------
inputdata = {}
validation = {}
exec_calculation = True

# 渡されたファイルの拡張子を確認
inputfile_name_split = os.path.splitext(inputfile_name)

if inputfile_name_split[-1] == ".xlsm":  # WEBPRO Ver2の入力シートであれば

  # jsonファイルの生成
  try:
    inputdata, validation = make_jsondata_from_Ver2_sheet(inputfile_name)

  except:
    validation = {
              "error": "入力シートの読み込み時に予期せぬエラーが発生しました。"
            }
    exec_calculation = False  # 計算は行わない。

In [56]:
    # 出力
with open(inputfile_name_split[0] + "_input.json",'w', encoding='utf-8') as fw:
  json.dump(inputdata, fw, indent=4, ensure_ascii=False, cls = MyEncoder)

In [57]:
#元々inputdataにPUMP, REFのキーはない，ここで宣言してる
inputdata["PUMP"] = {}
inputdata["REF"] = {}

# 計算結果を格納する変数
resultJson = {
    "設計一次エネルギー消費量[MJ/年]": 0,    # 空調設備の設計一次エネルギー消費量 [MJ/年]
    "基準一次エネルギー消費量[MJ/年]": 0,    # 空調設備の基準一次エネルギー消費量 [MJ/年]
    "設計一次エネルギー消費量[GJ/年]": 0,    # 空調設備の設計一次エネルギー消費量 [GJ/年]
    "基準一次エネルギー消費量[GJ/年]": 0,    # 空調設備の基準一次エネルギー消費量 [GJ/年]
    "設計一次エネルギー消費量[MJ/m2年]": 0,  # 空調設備の設計一次エネルギー消費量 [MJ/年]
    "基準一次エネルギー消費量[MJ/m2年]": 0,  # 空調設備の基準一次エネルギー消費量 [MJ/年]
    "計算対象面積": 0,
    "BEI/AC": 0,
    "Qroom": {
    },
    "AHU":{
    },
    "PUMP":{
    },
    "REF":{
    },
    "年間エネルギー消費量":{
        "空調機群ファン[MWh]": 0,     # 空調機群の一次エネルギー消費量 [MWh]
        "空調機群ファン[GJ]": 0,     # 空調機群の一次エネルギー消費量 [GJ]
        "空調機群全熱交換器[MWh]": 0,  # 全熱交換器の一次エネルギー消費量 [MWh]
        "空調機群全熱交換器[GJ]": 0,  # 全熱交換器の一次エネルギー消費量 [GJ]
        "二次ポンプ群[MWh]": 0,    # 二次ポンプ群の一次エネルギー消費量 [MWh]
        "二次ポンプ群[GJ]": 0,    # 二次ポンプ群の一次エネルギー消費量 [GJ]
        "熱源群熱源主機[MJ]": 0, # 熱源群主機の一次エネルギー消費量 [MJ]
        "熱源群熱源主機[GJ]": 0, # 熱源群主機の一次エネルギー消費量 [GJ]
        "熱源群熱源補機[MWh]": 0,   # 熱源群補機の一次エネルギー消費量 [MWh]
        "熱源群熱源補機[GJ]": 0,     # 熱源群補機の一次エネルギー消費量 [GJ]
        "熱源群一次ポンプ[MWh]": 0,   # 熱源群一次ポンプの一次エネルギー消費量 [MWh]
        "熱源群一次ポンプ[GJ]": 0,    # 熱源群一次ポンプの一次エネルギー消費量 [GJ]
        "熱源群冷却塔ファン[MWh]": 0,  # 熱源群冷却塔ファンの一次エネルギー消費量 [MWh]
        "熱源群冷却塔ファン[GJ]": 0,   # 熱源群冷却塔ファンの一次エネルギー消費量 [GJ]
        "熱源群冷却水ポンプ[MWh]": 0,  # 熱源群冷却水ポンプの一次エネルギー消費量 [MWh]
        "熱源群冷却水ポンプ[GJ]": 0,   # 熱源群冷却水ポンプの一次エネルギー消費量 [GJ]
    },
    "日別エネルギー消費量":{
        "E_fan_MWh_day": np.zeros(365),
        "E_pump_MWh_day": np.zeros(365),
        "E_ref_main_MWh_day": np.zeros(365),
        "E_ref_sub_MWh_day": np.zeros(365),
    },
    "Matrix":{
    },
    "for_CGS":{
    }
}

In [58]:
##----------------------------------------------------------------------------------
## 定数の設定
##----------------------------------------------------------------------------------
k_heatup = 0.84       # ファン・ポンプの発熱比率
Cw = 4.186            # 水の比熱 [kJ/kg・K]
divL = 11             # 負荷帯マトリックス分割数 （10区分＋過負荷1区分）
divT =  6             # 外気温度帯マトリックス分割数

##----------------------------------------------------------------------------------
## データベースファイルの読み込み
##----------------------------------------------------------------------------------

# 流量制御
with open(database_directory + 'FLOWCONTROL.json', 'r', encoding='utf-8') as f:
    FLOWCONTROL = json.load(f) #データファイルから生の配列に変換

# 熱源機器特性
with open(database_directory + "HeatSourcePerformance.json", 'r', encoding='utf-8') as f:
    HeatSourcePerformance = json.load(f) #上に同じ，熱源データ表

HeatSourcePerformance

{'温水ボイラ(灯油)': {'ID': 'HotwaterBoiler_Kerosene',
  '冷房時の特性': {'燃料種類': '',
   '熱源種類': '',
   '能力比': [],
   '入力比': [],
   '部分負荷特性': [],
   '送水温度特性': []},
  '暖房時の特性': {'燃料種類': '灯油',
   '熱源種類': '不要',
   '能力比': [{'下限': -1.0,
     '上限': 1.0,
     '冷却水温度下限': None,
     '冷却水温度上限': None,
     '係数': {'a4': 0.0, 'a3': 0.0, 'a2': 0.0, 'a1': 0.0, 'a0': 1.0},
     '基整促係数': 1.0}],
   '入力比': [{'下限': -1.0,
     '上限': 1.0,
     '冷却水温度下限': None,
     '冷却水温度上限': None,
     '係数': {'a4': 0.0, 'a3': 0.0, 'a2': 0.0, 'a1': 0.0, 'a0': 1.0},
     '基整促係数': 1.0}],
   '部分負荷特性': [{'下限': 0.1,
     '上限': 1.0,
     '冷却水温度下限': None,
     '冷却水温度上限': None,
     '係数': {'a4': 0.0, 'a3': 0.0, 'a2': 0.0, 'a1': 1.0, 'a0': 0.0},
     '基整促係数': 1.0}],
   '送水温度特性': [{'下限': -1.0,
     '上限': 1.0,
     '冷却水温度下限': None,
     '冷却水温度上限': None,
     '係数': {'a4': 0.0, 'a3': 0.0, 'a2': 0.0, 'a1': 0.0, 'a0': 1.0},
     '基整促係数': 1.0}]}},
 'パッケージエアコンディショナ(水冷式熱回収形)': {'ID': 'PackagedAirConditioner_WaterSource_HR',
  '冷房時の特性': {'燃料種類': '電力',
   

In [59]:
##----------------------------------------------------------------------------------
## 任意評定 （SP-1: 流量制御)
##----------------------------------------------------------------------------------

# 任意評定用の入力があれば追加，基本ないかも？
if "SpecialInputData" in inputdata:
    if "flow_control" in inputdata["SpecialInputData"]:
        FLOWCONTROL.update(inputdata["SpecialInputData"]["flow_control"])

##----------------------------------------------------------------------------------
## 任意評定 （SP-2：　熱源機器特性)
##----------------------------------------------------------------------------------

# 任意評定用の入力があれば追加
if "SpecialInputData" in inputdata: #SpecialInputDataはなんか空だけど入ってる
    if "heatsource_performance" in inputdata["SpecialInputData"]:
        HeatSourcePerformance.update(inputdata["SpecialInputData"]["heatsource_performance"])


##----------------------------------------------------------------------------------
## マトリックスの設定
##----------------------------------------------------------------------------------

# 地域別データの読み込み
with open(database_directory + 'AREA.json', 'r', encoding='utf-8') as f:
    Area = json.load(f) #Areaに入っているのは，地域区分と使用する代表点の気象データ，2.2.1の表3

In [60]:
Area[inputdata["Building"]["Region"]+"地域"]["暖房時外気温下限"]
Area
inputdata["Building"]["Region"]
# inputdata["Building"]
# divL
# inputdata["Building"]["Region"]

inputdata["AirConditioningZone"]

{'1F_ロビー': {'isNatualVentilation': '無',
  'isSimultaneousSupply': '無',
  'AHU_cooling_insideLoad': 'FCU1-1',
  'AHU_cooling_outdoorLoad': 'FCU1-1',
  'AHU_heating_insideLoad': 'FCU1-1',
  'AHU_heating_outdoorLoad': 'FCU1-1',
  'Info': None},
 '1F_EVホール': {'isNatualVentilation': '無',
  'isSimultaneousSupply': '無',
  'AHU_cooling_insideLoad': 'FCU1-2',
  'AHU_cooling_outdoorLoad': 'FCU1-2',
  'AHU_heating_insideLoad': 'FCU1-2',
  'AHU_heating_outdoorLoad': 'FCU1-2',
  'Info': None},
 '1F_中央監視室・警備室': {'isNatualVentilation': '無',
  'isSimultaneousSupply': '無',
  'AHU_cooling_insideLoad': 'EHP1-1',
  'AHU_cooling_outdoorLoad': 'EHP1-1',
  'AHU_heating_insideLoad': 'EHP1-1',
  'AHU_heating_outdoorLoad': 'EHP1-1',
  'Info': None},
 '1F_更衣室1': {'isNatualVentilation': '無',
  'isSimultaneousSupply': '無',
  'AHU_cooling_insideLoad': 'EHP1-2',
  'AHU_cooling_outdoorLoad': 'EHP1-2',
  'AHU_heating_insideLoad': 'EHP1-2',
  'AHU_heating_outdoorLoad': 'EHP1-2',
  'Info': None},
 '1F_更衣室2': {'isNatualV

In [61]:
# 負荷率帯マトリックス mxL = array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.2])
#記号はL
mxL = np.arange(1/(divL-1), 1.01, 1/(divL-1))
mxL = np.append(mxL,1.2)

# 負荷率帯マトリックス（平均） aveL = array([0.05, 0.15, 0.25, 0.35, 0.45, 0.55, 0.65, 0.75, 0.85, 0.95, 1.2 ])
aveL = np.zeros(len(mxL))
for iL in range(0,len(mxL)):
    if iL == 0:
        aveL[0] = mxL[0]/2
    elif iL == len(mxL)-1:
        aveL[iL] = 1.2
    else:
        aveL[iL] = mxL[iL-1] + (mxL[iL]-mxL[iL-1])/2

##----------------------------------------------------------------------------------
## 日平均外気温 （解説書 2.7.4.1）
##----------------------------------------------------------------------------------

# 外気温度帯の上限・下限, 日射地域区分のRegionをもとにAreaデータベースを参照，冷暖房期の外気温下限上限の地域値を読み込んでくる．
mxTH_min = Area[inputdata["Building"]["Region"]+"地域"]["暖房時外気温下限"]
mxTH_max = Area[inputdata["Building"]["Region"]+"地域"]["暖房時外気温上限"]
mxTC_min = Area[inputdata["Building"]["Region"]+"地域"]["冷房時外気温下限"]
mxTC_max = Area[inputdata["Building"]["Region"]+"地域"]["冷房時外気温上限"]

delTC = (mxTC_max - mxTC_min)/divT
delTH = (mxTH_max - mxTH_min)/divT

#冷暖房時の外気温をdivT（=6）で線形分割，array化.Cは冷房，Hは暖房の意味
mxTC = np.arange(mxTC_min+delTC, mxTC_max+delTC, delTC)
mxTH = np.arange(mxTH_min+delTH, mxTH_max+delTH, delTH)

ToadbC = mxTC - delTC/2
ToadbH = mxTH - delTH/2


In [62]:
#サンプルではclimate.readHaspClimateDataで，６地域に書かれてるC1_6158195.hasを読みに行ってる
# Toutは外気温[°C], Xoutは外気絶対湿度 [kg/kgDA],Iodは法線面直達日射量 [W/m2], Iosは水平面天空日射量 [W/m2], Innは水平面夜間放射量 [W/m2]
[ToutALL, XoutALL, IodALL, IosALL, InnALL] = climate.readHaspClimateData( climatedata_directory + "/C1_" + Area[inputdata["Building"]["Region"]+"地域"]["気象データファイル名"] )
#それぞれサイズは
ToutALL.shape
# print((np.mean(XoutALL[:,[6,7,8,9,10,11,12,13,14,15,16,17]],1)))　←これは動いている時間の湿度を表示しているということか

(365, 24)

In [63]:
# 緯度
phi  = Area[inputdata["Building"]["Region"]+"地域"]["緯度"]
# 経度
longi  = Area[inputdata["Building"]["Region"]+"地域"]["経度"]

##----------------------------------------------------------------------------------
## 冷暖房期間（解説書 2.2.2）
##----------------------------------------------------------------------------------

# 空調運転モード,2.2.2の表6
with open(database_directory + 'ACoperationMode.json', 'r', encoding='utf-8') as f:
    ACoperationMode = json.load(f)

# 各日の冷暖房期間の種類（冷房期、暖房期、中間期）（365×1の行列）, 6地域だと'温暖地'が選択される
ac_mode = ACoperationMode[ Area[inputdata["Building"]["Region"]+"地域"]["空調運転モードタイプ"] ]

# ac_mode #日付ごとの冷暖房日，仕様書のSeason_dに相当するリスト

In [64]:
##----------------------------------------------------------------------------------
## 平均外気温（解説書 2.2.3）
##----------------------------------------------------------------------------------
#aveはその日のただの平均，dayは昼間，ngtは夜間，年間平均外気温とか冷暖房時の平均外気温
# 日平均外気温[℃]（365×1）
Toa_ave = np.mean(ToutALL,1)
Toa_day = np.mean(ToutALL[:,[6,7,8,9,10,11,12,13,14,15,16,17]],1)
Toa_ngt = np.mean(ToutALL[:,[0,1,2,3,4,5,18,19,20,21,22,23]],1)

# 日平均外気絶対湿度 [kg/kgDA]（365×1）
Xoa_ave = np.mean(XoutALL,1)
Xoa_day = np.mean(XoutALL[:,[6,7,8,9,10,11,12,13,14,15,16,17]],1)
Xoa_ngt = np.mean(XoutALL[:,[0,1,2,3,4,5,18,19,20,21,22,23]],1)


##----------------------------------------------------------------------------------
## 外気エンタルピー（解説書 2.2.4）
##----------------------------------------------------------------------------------

Hoa_ave = air_enthalpy(Toa_ave, Xoa_ave)
Hoa_day = air_enthalpy(Toa_day, Xoa_day)
Hoa_ngt = air_enthalpy(Toa_ngt, Xoa_ngt)


##----------------------------------------------------------------------------------
## 空調室の設定温度、室内エンタルピー（解説書 2.3.1、2.3.2）
##----------------------------------------------------------------------------------

#配列の各要素に各日付でのデータを入れていく，温度/室内エンタルピーの定値は仕様書表12/14の下にかかれているやつをベタ打ち
TroomSP = np.zeros(365)    # 室内設定温度
RroomSP = np.zeros(365)    # 室内設定湿度
Hroom   = np.zeros(365)    # 室内設定エンタルピー

for dd in range(0,365):

    if ac_mode[dd] == "冷房":
        TroomSP[dd] = 26
        RroomSP[dd] = 50
        Hroom[dd] = 52.91

    elif ac_mode[dd] == "中間":
        TroomSP[dd] = 24
        RroomSP[dd] = 50
        Hroom[dd] = 47.81

    elif ac_mode[dd] == "暖房":
        TroomSP[dd] = 22
        RroomSP[dd] = 40
        Hroom[dd] = 38.81


In [90]:
##----------------------------------------------------------------------------------
## 空調機の稼働状態、内部発熱量（解説書 2.3.3、2.3.4）
##----------------------------------------------------------------------------------

#ここに関してはややこしいdatabaseから引っ張ってこないといけない．ルールが複雑．．．スケジュールってなんや

input_calendar = []

roomAreaTotal = 0
roomScheduleRoom   = {} #室同時使用率
roomScheduleLight  = {} #照明発熱密度比率
roomSchedulePerson = {} #人体発熱密度比率
roomScheduleOAapp  = {} #機器発熱密度比率
roomDayMode        = {} #'1F_ロビー': '昼'とかそういう情報が格納される，これだけ室の数と同じ長さのベクトルになる

# 空調ゾーン毎にループ
for room_zone_name in inputdata["AirConditioningZone"]:

    if room_zone_name in inputdata["Rooms"]:  # ゾーン分けがない場合

        # 建物用途・室用途、ゾーン面積等の取得
        # AirConditioningZoneのデータシートにbuildingTypeやroomTypeなどをappend
        # buildingTypeは事務所等とか，roomTypeはロビーとかAC情報と同じ2-1)シートにあるけど分割してデータフレーム化しているから適宜読み出してappendしている
        inputdata["AirConditioningZone"][room_zone_name]["buildingType"]  = inputdata["Rooms"][room_zone_name]["buildingType"]
        inputdata["AirConditioningZone"][room_zone_name]["roomType"]      = inputdata["Rooms"][room_zone_name]["roomType"]
        inputdata["AirConditioningZone"][room_zone_name]["zoneArea"]      = inputdata["Rooms"][room_zone_name]["roomArea"]
        inputdata["AirConditioningZone"][room_zone_name]["ceilingHeight"] = inputdata["Rooms"][room_zone_name]["ceilingHeight"]

    # print(room_zone_name)

    print(inputdata["AirConditioningZone"][room_zone_name]["roomType"])
    # print(input_calendar)

    # 365日×24時間分のスケジュール （365×24の行列を格納した dict型）
    # get_roomUsageSchedule(buildingType, roomType, input_calendar={})buildtypeとroomtypeが決まればスケジュールは決まる
    #カレンダーパターンコードはA~F, カレンダーパターンは1,2,3のどれか
    roomScheduleRoom[room_zone_name], roomScheduleLight[room_zone_name], roomSchedulePerson[room_zone_name], roomScheduleOAapp[room_zone_name], roomDayMode[room_zone_name] = \
        bc.get_roomUsageSchedule(inputdata["AirConditioningZone"][room_zone_name]["buildingType"], inputdata["AirConditioningZone"][room_zone_name]["roomType"], input_calendar)

    # print(inputdata["AirConditioningZone"][room_zone_name]["buildingType"], inputdata["AirConditioningZone"][room_zone_name]["roomType"])

    # 空調対象面積の合計
    roomAreaTotal += inputdata["AirConditioningZone"][room_zone_name]["zoneArea"]


ロビー
廊下
中央監視室
更衣室又は倉庫
更衣室又は倉庫
更衣室又は倉庫
事務室
事務室
廊下
事務室
事務室
廊下
事務室
事務室
廊下
事務室
事務室
廊下
事務室
事務室
廊下
事務室
事務室
廊下
事務室
事務室


In [96]:
# inputdata["AirConditioningZone"]["事務所等"]["buildingType"]
inputdata["Rooms"]
# bc.get_roomUsageSchedule("事務所等", "事務室", [])
# len(inputdata["AirConditioningZone"])

{'1F_風除け室': {'buildingType': '事務所等',
  'roomType': '廊下',
  'floorHeight': 5.0,
  'ceilingHeight': 2.6,
  'roomArea': 21.12,
  'zone': None,
  'modelBuildingType': '事務所モデル',
  'buildingGroup': None,
  'Info': None},
 '1F_ロビー': {'buildingType': '事務所等',
  'roomType': 'ロビー',
  'floorHeight': 5.0,
  'ceilingHeight': 3.5,
  'roomArea': 114.12,
  'zone': None,
  'modelBuildingType': '事務所モデル',
  'buildingGroup': None,
  'Info': None},
 '1F_EVホール': {'buildingType': '事務所等',
  'roomType': '廊下',
  'floorHeight': 5.0,
  'ceilingHeight': 3.5,
  'roomArea': 16.32,
  'zone': None,
  'modelBuildingType': '事務所モデル',
  'buildingGroup': None,
  'Info': None},
 '1F_中央監視室・警備室': {'buildingType': '事務所等',
  'roomType': '中央監視室',
  'floorHeight': 5.0,
  'ceilingHeight': 2.6,
  'roomArea': 39.0,
  'zone': None,
  'modelBuildingType': '事務所モデル',
  'buildingGroup': None,
  'Info': None},
 '1F_更衣室1': {'buildingType': '事務所等',
  'roomType': '更衣室又は倉庫',
  'floorHeight': 5.0,
  'ceilingHeight': 2.4,
  'roomArea': 14.63,
  

In [66]:

##----------------------------------------------------------------------------------
## 室負荷計算（解説書 2.4）
##----------------------------------------------------------------------------------

for room_zone_name in inputdata["AirConditioningZone"]:

    resultJson["Qroom"][room_zone_name] = {
        "Qwall_T": np.zeros(365),  # 壁からの温度差による熱取得 [W/m2]
        "Qwall_S": np.zeros(365),  # 壁からの日射による熱取得 [W/m2]
        "Qwall_N": np.zeros(365),  # 壁からの夜間放射による熱取得（マイナス）[W/m2]
        "Qwind_T": np.zeros(365),  # 窓からの温度差による熱取得 [W/m2]
        "Qwind_S": np.zeros(365),  # 窓からの日射による熱取得 [W/m2]
        "Qwind_N": np.zeros(365),  # 窓からの夜間放射による熱取得（マイナス）[W/m2]
        "QroomDc": np.zeros(365),  # 冷房熱取得（日積算）　[MJ/day]
        "QroomDh": np.zeros(365),   # 暖房熱取得（日積算）　[MJ/day]
        "QroomHc": np.zeros((365,24)),  # 冷房熱取得（時刻別）　[MJ/h]
        "QroomHh": np.zeros((365,24))   # 暖房熱取得（時刻別）　[MJ/h]
    }

##----------------------------------------------------------------------------------
## 外皮面への入射日射量（解説書 2.4.1）
##----------------------------------------------------------------------------------

solor_radiation = {
    "直達":{
    },
    "直達_入射角特性込":{
    },
    "天空":{
    },
    "夜間":{
    }
}

# 方位角別の日射量(仕様書2.4.1 表2.4.1の計算)
# solarRadiationByAzimuth(alp, bet, phi, longi, IodALL, IosALL, InnALL)関数の雛形は左記の通り
# Idsr, jd~Insr,j,dまで日射の各方位dに対して計算していく
# 方位d = [南，　南西，　西，　北西，　北，　北東，　東，　南東，　水平]， Output = [Idsr_j_d, I'dsr_j_d, Iisr_j_d, Insr_j_d]
# Idsr, I'dsrの違いはetaの影響が入ってるかどうか．
(solor_radiation["直達"]["南"],  solor_radiation["直達_入射角特性込"]["南"], solor_radiation["天空"]["垂直"], solor_radiation["夜間"]["垂直"])  = climate.solarRadiationByAzimuth(  0, 90, phi, longi, IodALL, IosALL, InnALL)
(solor_radiation["直達"]["南西"], solor_radiation["直達_入射角特性込"]["南西"], _, _) = climate.solarRadiationByAzimuth( 45, 90, phi, longi, IodALL, IosALL, InnALL)
(solor_radiation["直達"]["西"],  solor_radiation["直達_入射角特性込"]["西"], _, _)  = climate.solarRadiationByAzimuth( 90, 90, phi, longi, IodALL, IosALL, InnALL)
(solor_radiation["直達"]["北西"], solor_radiation["直達_入射角特性込"]["北西"], _, _) = climate.solarRadiationByAzimuth(135, 90, phi, longi, IodALL, IosALL, InnALL)
(solor_radiation["直達"]["北"],  solor_radiation["直達_入射角特性込"]["北"], _, _)  = climate.solarRadiationByAzimuth(180, 90, phi, longi, IodALL, IosALL, InnALL)
(solor_radiation["直達"]["北東"], solor_radiation["直達_入射角特性込"]["北東"], _, _) = climate.solarRadiationByAzimuth(225, 90, phi, longi, IodALL, IosALL, InnALL)
(solor_radiation["直達"]["東"],  solor_radiation["直達_入射角特性込"]["東"], _, _)  = climate.solarRadiationByAzimuth(270, 90, phi, longi, IodALL, IosALL, InnALL)
(solor_radiation["直達"]["南東"], solor_radiation["直達_入射角特性込"]["南東"], _, _) = climate.solarRadiationByAzimuth(315, 90, phi, longi, IodALL, IosALL, InnALL)
(solor_radiation["直達"]["水平"], solor_radiation["直達_入射角特性込"]["水平"], solor_radiation["天空"]["水平"], solor_radiation["夜間"]["水平"])  = climate.solarRadiationByAzimuth(  0,  0, phi, longi, IodALL, IosALL, InnALL)

In [67]:
## Solor radiation計算の一例テストセクション
# (solor_radiation["直達"]["南"],  solor_radiation["直達_入射角特性込"]["南"], solor_radiation["天空"]["垂直"], solor_radiation["夜間"]["垂直"])  = climate.solarRadiationByAzimuth(  0, 90, phi, longi, IodALL, IosALL, InnALL)
alp, bet, phi, longi, IodALL, IosALL, InnALL = [0, 90, phi, longi, IodALL, IosALL, InnALL]

# 度からラジアンへの変換係数
rad = math.pi / 180

# 各月の日数
monthly_daynum = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]

go = 1 #なんでこんなもんがいる？

# 初期化
Id     = np.zeros((365,24))
Id_ita = np.zeros((365,24))
Is     = np.zeros((365,24))
ita    = np.zeros((365,24))

# 通算日数(1月1日が1、12月31日が365)
DN = 0

for month in range(1,13):
    for day in range(1,monthly_daynum[month-1]+1):

        for hour in range(0,24):

            # 日射量 [W/m2]
            Iod  = IodALL[DN,hour] # 法線面直達日射量 [W/m2]
            Ios  = IosALL[DN,hour] # 水平面天空日射量 [W/m2]
            Ion  = InnALL[DN,hour] # 水平面夜間放射量 [W/m2]

            # 中央標準時を求める
            t = (hour+1) + 0 / 60
            # 日赤緯を求める(HASP教科書P24(2-22)参照)
            # climate.pyのdel04関数の計算をベタ打ち
            # 通日を計算する
            n = math.floor( 30*(month-1) + math.floor((month+math.floor(month/8))/2) - math.floor((month+7)/10) + day)
            # 一年を周期とする角度を計算する
            w = n * 2.0 * math.pi / 366.0

            # HASP教科書等より2項目の括弧内の2項目の係数を変更（+0.2070988 → -0.2070988）
            declination = 0.006322 - 0.405748 * math.cos(w + 0.153231) - 0.005880 * math.cos(2.0 * w - 0.207099) - 0.003233 * math.cos(3 * w + 0.620129)

            # 均時差を求める
            equal_time_difference = - 0.0002786409 + 0.1227715 * math.cos(w + 1.498311) - 0.1654575 * math.cos(2.0 * w - 1.261546) - 0.00535383 * math.cos(3.0 * w -1.1571)
            # 時角を求める
            Tim = (15.0 * t + 15.0 * equal_time_difference + longi - 315.0) * rad

            sinPhi = math.sin(phi * rad) # 緯度の正弦
            cosPhi = math.cos(phi * rad) # 緯度の余弦
            sinAlp = math.sin(alp * rad)    # 方位角正弦
            cosAlp = math.cos(alp * rad)    # 方位角余弦
            sinBet = math.sin(bet * rad)    # 傾斜角正弦
            cosBet = math.cos(bet * rad)    # 傾斜角余弦
            sinDel = math.sin(declination)          # 日赤緯の正弦
            cosDel = math.cos(declination)          # 日赤緯の余弦
            sinTim = math.sin(Tim)          # 時角の正弦
            cosTim = math.cos(Tim)          # 時角の余弦

            # 太陽高度の正弦を求める(HASP教科書 P25 (2.25)参照 )
            sinh   = sinPhi * sinDel + cosPhi * cosDel * cosTim

            # 太陽高度の余弦、太陽方位の正弦・余弦を求める(HASP 教科書P25 (2.25)参照)
            cosh   = math.sqrt(1 - sinh**2)                     # 太陽高度の余弦
            sinA   = cosDel * sinTim / cosh                     # 太陽方位の正弦
            cosA   = (sinh * sinPhi - sinDel)/(cosh * cosPhi)   # 太陽方位の余弦

            # 傾斜壁から見た太陽高度を求める(HASP 教科書 P26(2.26)参照)
            # sinh2はSdsrのこと
            sinh2  = sinh * cosBet + cosh * sinBet * (cosA * cosAlp + sinA * sinAlp)

            if sinh2 < 0:
                sinh2 = 0

            # 入射角特性
            ita[DN,hour] = 2.392 * sinh2 - 3.8636 * sinh2**3 + 3.7568 * sinh2**5 - 1.3952 * sinh2**7
            # print(ita[DN,hour])

            # 傾斜面入射日射量(直達日射量)（W/m2）
            Id[DN,hour]  = go * Iod * sinh2

            # 傾斜面入射日射量(直達日射量)（W/m2）　入射角特性込み（0.89で除して基準化済み）， (ita)max = 0.89 (when sinh2=1)なのか？
            Id_ita[DN,hour] = go * Iod * sinh2 * ita[DN,hour]/0.89

            # 傾斜面入射日射量(天空日射量)（W/m2）, Iisr
            if bet == 90:
                Is[DN,hour] = 0.5*Ios + 0.1*0.5*(Ios + Iod*sinh)
            elif bet == 0:
                Is[DN,hour] = Ios
            else:
                # 太陽熱利用の計算用：要検証
                rhoG = 0.8
                Is[DN,hour] = (1+cosBet)/2*Ios + (1-cosBet)/2*rhoG*(Ios + Iod*sinh) #これなに


        # 日数カウント
        DN += 1


# 長波長放射
if bet == 90:
    Insr = np.sum(InnALL,1)/2
elif bet == 0:
    Insr = np.sum(InnALL,1)
else:
    Insr = None

print(np.sum(Id,1), np.sum(Id_ita,1), np.sum(Is,1), Insr) #垂直，入射角込，天空，長波長放射


[3.22052822e+03 9.44742904e+02 8.61567892e+01 1.25002138e+02
 2.21727315e+03 4.43900986e+03 3.50575011e+02 4.47877753e+03
 3.06922425e+03 4.25916859e+03 2.72293453e+03 4.11990252e+03
 4.58956749e+03 3.35485686e+03 2.37694377e+03 2.33188707e+03
 5.83486062e+02 5.75196431e+02 7.95985643e+02 1.25695875e+03
 3.94792705e+03 9.33105931e+01 9.82176404e+01 1.93155894e+03
 7.52149778e+02 1.84198851e+03 2.37415097e+03 2.42216175e+03
 3.32201019e+03 4.49964115e+03 3.40215904e+03 2.22567033e+03
 2.55493695e+03 3.38303022e+03 2.72297261e+03 1.28860961e+03
 1.71542574e+03 3.97802330e+03 2.04334578e+03 2.75346529e+03
 7.88268240e+01 3.57905211e+02 2.87544538e+03 6.86346923e+02
 1.07075483e+02 9.54075352e+01 1.17854018e+03 1.68559351e+03
 5.78887946e+02 3.03536257e+03 1.82242555e+03 2.47496335e+03
 2.08937419e+03 3.01945362e+03 3.73201530e+03 2.00568319e+03
 4.62997793e+03 3.39642646e+03 4.65467151e+01 5.22587374e+01
 3.05392348e+03 5.42713762e+02 2.15291639e+03 3.86180664e+03
 5.73936422e+02 3.973515

In [68]:

##----------------------------------------------------------------------------------
## 外壁等の熱貫流率の算出（解説書 附属書A.1）
##----------------------------------------------------------------------------------

### ISSUE : 二つのデータベースにわかれてしまっているので統一する。###

# 標準入力法建材データの読み込み
with open(database_directory + 'HeatThermalConductivity.json', 'r', encoding='utf-8') as f:
    HeatThermalConductivity = json.load(f)

# モデル建物法建材データの読み込み
with open(database_directory + 'HeatThermalConductivity_model.json', 'r', encoding='utf-8') as f:
    HeatThermalConductivity_model = json.load(f)

if "WallConfigure" in inputdata:  # WallConfigure があれば以下を実行

    for wall_name in inputdata["WallConfigure"].keys():

        if inputdata["WallConfigure"][wall_name]["inputMethod"] == "断熱材種類を入力":

            if inputdata["WallConfigure"][wall_name]["materialID"] == "無": # 断熱材種類が「無」の場合

                inputdata["WallConfigure"][wall_name]["Uvalue_wall"]  = 2.63
                inputdata["WallConfigure"][wall_name]["Uvalue_roof"]  = 1.53
                inputdata["WallConfigure"][wall_name]["Uvalue_floor"] = 2.67

            else: # 断熱材種類が「無」以外、もしくは、熱伝導率が直接入力されている場合

                # 熱伝導率の指定がない場合は「断熱材種類」から推定
                if (inputdata["WallConfigure"][wall_name]["conductivity"] == None):

                    inputdata["WallConfigure"][wall_name]["conductivity"] = \
                        float( HeatThermalConductivity_model[ inputdata["WallConfigure"][wall_name]["materialID"] ] )

                # 熱伝導率と厚みとから、熱貫流率を計算（３種類）
                inputdata["WallConfigure"][wall_name]["Uvalue_wall"] = \
                    0.663 * (inputdata["WallConfigure"][wall_name]["thickness"] / 1000 / inputdata["WallConfigure"][wall_name]["conductivity"]) ** (-0.638)
                inputdata["WallConfigure"][wall_name]["Uvalue_roof"] = \
                    0.548 * (inputdata["WallConfigure"][wall_name]["thickness"] / 1000 / inputdata["WallConfigure"][wall_name]["conductivity"]) ** (-0.524)
                inputdata["WallConfigure"][wall_name]["Uvalue_floor"] = \
                    0.665 * (inputdata["WallConfigure"][wall_name]["thickness"] / 1000 / inputdata["WallConfigure"][wall_name]["conductivity"]) ** (-0.641)


        elif inputdata["WallConfigure"][wall_name]["inputMethod"] == "建材構成を入力":

            Rvalue = 0.11 + 0.04

            for layer in enumerate(inputdata["WallConfigure"][wall_name]["layers"]):

                # 熱伝導率が空欄である場合、建材名称から熱伝導率を見出す。
                if layer[1]["conductivity"] == None:

                    if (layer[1]["materialID"] == "密閉中空層") or (layer[1]["materialID"] == "非密閉中空層"):

                        # 空気層の場合
                        Rvalue += HeatThermalConductivity[layer[1]["materialID"]]["熱抵抗値"]

                    else:

                        # 空気層以外の断熱材を指定している場合
                        if layer[1]["thickness"] != None:
                            material_name = layer[1]["materialID"].replace('\u3000', '')
                            Rvalue += (layer[1]["thickness"]/1000) / HeatThermalConductivity[material_name]["熱伝導率"]

                else:

                    # 熱伝導率を入力している場合
                    Rvalue += (layer[1]["thickness"]/1000) / layer[1]["conductivity"]

            inputdata["WallConfigure"][wall_name]["Uvalue"] = 1/Rvalue


In [69]:
inputdata["WallConfigure"][wall_name]["layers"][1]
# inputdata["WallConfigure"][wall_name]["inputMethod"]

{'materialID': 'セメント・モルタル',
 'conductivity': None,
 'thickness': 27.0,
 'Info': None}

In [70]:
inputdata["EnvelopeSet"][room_zone_name]["WallList"]

[{'Direction': '南',
  'EnvelopeArea': 159.2,
  'EnvelopeWidth': None,
  'EnvelopeHeight': None,
  'WallSpec': 'W1',
  'WallType': '日の当たる外壁',
  'WindowList': [{'WindowID': 'G1',
    'WindowNumber': 26.6,
    'isBlind': '有',
    'EavesID': '無',
    'Info': None}]},
 {'Direction': '西',
  'EnvelopeArea': 60.0,
  'EnvelopeWidth': None,
  'EnvelopeHeight': None,
  'WallSpec': 'W1',
  'WallType': '日の当たる外壁',
  'WindowList': [{'WindowID': 'G1',
    'WindowNumber': 10.64,
    'isBlind': '有',
    'EavesID': '無',
    'Info': None}]},
 {'Direction': '水平（下）',
  'EnvelopeArea': 499.5,
  'EnvelopeWidth': None,
  'EnvelopeHeight': None,
  'WallSpec': 'R1',
  'WallType': '日の当たる外壁',
  'WindowList': [{'WindowID': '無',
    'WindowNumber': None,
    'isBlind': '無',
    'EavesID': '無',
    'Info': None}]}]

In [71]:

##----------------------------------------------------------------------------------
## 窓の熱貫流率及び日射熱取得率の算出（解説書 附属書A.2）
##----------------------------------------------------------------------------------

# 窓データの読み込み
with open(database_directory + 'WindowHeatTransferPerformance.json', 'r', encoding='utf-8') as f:
    WindowHeatTransferPerformance = json.load(f)

with open(database_directory + 'glass2window.json', 'r', encoding='utf-8') as f:
    glass2window = json.load(f)


if "WindowConfigure" in inputdata:

    for window_name in inputdata["WindowConfigure"].keys():

        if inputdata["WindowConfigure"][window_name]["inputMethod"] == "ガラスの種類を入力":

            # 建具の種類の読み替え
            if inputdata["WindowConfigure"][window_name]["frameType"] == "木製" or \
                inputdata["WindowConfigure"][window_name]["frameType"] == "樹脂製":

                inputdata["WindowConfigure"][window_name]["frameType"] = "木製・樹脂製建具"

            elif inputdata["WindowConfigure"][window_name]["frameType"] == "金属木複合製" or \
                inputdata["WindowConfigure"][window_name]["frameType"] == "金属樹脂複合製":

                inputdata["WindowConfigure"][window_name]["frameType"] = "金属木複合製・金属樹脂複合製建具"

            elif inputdata["WindowConfigure"][window_name]["frameType"] == "金属製":

                inputdata["WindowConfigure"][window_name]["frameType"] = "金属製建具"


            # ガラスIDと建具の種類から、熱貫流率・日射熱取得率を抜き出す。
            inputdata["WindowConfigure"][window_name]["Uvalue"] = \
            WindowHeatTransferPerformance\
                [ inputdata["WindowConfigure"][window_name]["glassID"] ]\
                [ inputdata["WindowConfigure"][window_name]["frameType"] ]["熱貫流率"]

            inputdata["WindowConfigure"][window_name]["Uvalue_blind"] = \
            WindowHeatTransferPerformance\
                [ inputdata["WindowConfigure"][window_name]["glassID"] ]\
                [ inputdata["WindowConfigure"][window_name]["frameType"] ]["熱貫流率・ブラインド込"]

            inputdata["WindowConfigure"][window_name]["Ivalue"] = \
            WindowHeatTransferPerformance\
                [ inputdata["WindowConfigure"][window_name]["glassID"] ]\
                [ inputdata["WindowConfigure"][window_name]["frameType"] ]["日射熱取得率"]

            inputdata["WindowConfigure"][window_name]["Ivalue_blind"] = \
            WindowHeatTransferPerformance\
                [ inputdata["WindowConfigure"][window_name]["glassID"] ]\
                [ inputdata["WindowConfigure"][window_name]["frameType"] ]["日射熱取得率・ブラインド込"]


        elif inputdata["WindowConfigure"][window_name]["inputMethod"] == "ガラスの性能を入力":

            ku_a = 0
            ku_b = 0
            kita  = 0
            dR = 0

            # 建具の種類の読み替え
            if inputdata["WindowConfigure"][window_name]["frameType"] == "木製" or \
                inputdata["WindowConfigure"][window_name]["frameType"] == "樹脂製":

                inputdata["WindowConfigure"][window_name]["frameType"] = "木製・樹脂製建具"

            elif inputdata["WindowConfigure"][window_name]["frameType"] == "金属木複合製" or \
                inputdata["WindowConfigure"][window_name]["frameType"] == "金属樹脂複合製":

                inputdata["WindowConfigure"][window_name]["frameType"] = "金属木複合製・金属樹脂複合製建具"

            elif inputdata["WindowConfigure"][window_name]["frameType"] == "金属製":

                inputdata["WindowConfigure"][window_name]["frameType"] = "金属製建具"


            # 変換係数
            ku_a = glass2window[inputdata["WindowConfigure"][window_name]["frameType"]][inputdata["WindowConfigure"][window_name]["layerType"]]["ku_a1"] \
                / glass2window[inputdata["WindowConfigure"][window_name]["frameType"]][inputdata["WindowConfigure"][window_name]["layerType"]]["ku_a2"]
            ku_b = glass2window[inputdata["WindowConfigure"][window_name]["frameType"]][inputdata["WindowConfigure"][window_name]["layerType"]]["ku_b1"] \
                / glass2window[inputdata["WindowConfigure"][window_name]["frameType"]][inputdata["WindowConfigure"][window_name]["layerType"]]["ku_b2"]
            kita  = glass2window[inputdata["WindowConfigure"][window_name]["frameType"]][inputdata["WindowConfigure"][window_name]["layerType"]]["kita"]

            # print(ku_a)
            # print(ku_b)
            # print(glass2window[inputdata["WindowConfigure"][window_name]["frameType"]][inputdata["WindowConfigure"][window_name]["layerType"]]["ku_a1"] )
            # print(glass2window[inputdata["WindowConfigure"][window_name]["frameType"]][inputdata["WindowConfigure"][window_name]["layerType"]]["ku_a2"] )
            # print(glass2window[inputdata["WindowConfigure"][window_name]["frameType"]][inputdata["WindowConfigure"][window_name]["layerType"]]["ku_b1"] )
            # print(glass2window[inputdata["WindowConfigure"][window_name]["frameType"]][inputdata["WindowConfigure"][window_name]["layerType"]]["ku_b2"] )
            # print(inputdata["WindowConfigure"][window_name]["glassUvalue"])

            inputdata["WindowConfigure"][window_name]["Uvalue"] = ku_a * inputdata["WindowConfigure"][window_name]["glassUvalue"] + ku_b
            inputdata["WindowConfigure"][window_name]["Ivalue"] = kita * inputdata["WindowConfigure"][window_name]["glassIvalue"]

            # ガラスの熱貫流率と日射熱取得率が入力されている場合は、ブラインドの効果を見込む
            dR = (0.021 / inputdata["WindowConfigure"][window_name]["glassUvalue"]) + 0.022

            inputdata["WindowConfigure"][window_name]["Uvalue_blind"] = \
                1 / ( ( 1/inputdata["WindowConfigure"][window_name]["Uvalue"]) + dR )

            inputdata["WindowConfigure"][window_name]["Ivalue_blind"] = \
                inputdata["WindowConfigure"][window_name]["Ivalue"] / inputdata["WindowConfigure"][window_name]["glassIvalue"] \
                    * (-0.1331 * inputdata["WindowConfigure"][window_name]["glassIvalue"] ** 2 +\
                            0.8258 * inputdata["WindowConfigure"][window_name]["glassIvalue"] )


        elif inputdata["WindowConfigure"][window_name]["inputMethod"] == "性能値を入力":

            inputdata["WindowConfigure"][window_name]["Uvalue"] = inputdata["WindowConfigure"][window_name]["windowUvalue"]
            inputdata["WindowConfigure"][window_name]["Ivalue"] = inputdata["WindowConfigure"][window_name]["windowIvalue"]

            # ブラインド込みの値を計算
            dR = 0

            if inputdata["WindowConfigure"][window_name]["glassUvalue"] == None or \
                inputdata["WindowConfigure"][window_name]["glassIvalue"] == None:

                inputdata["WindowConfigure"][window_name]["Uvalue_blind"] = inputdata["WindowConfigure"][window_name]["windowUvalue"]
                inputdata["WindowConfigure"][window_name]["Ivalue_blind"] = inputdata["WindowConfigure"][window_name]["windowIvalue"]

            else:
                # ガラスの熱貫流率と日射熱取得率が入力されている場合は、ブラインドの効果を見込む
                dR = (0.021 / inputdata["WindowConfigure"][window_name]["glassUvalue"]) + 0.022

                inputdata["WindowConfigure"][window_name]["Uvalue_blind"] = \
                    1 / ( ( 1/inputdata["WindowConfigure"][window_name]["windowUvalue"]) + dR )

                inputdata["WindowConfigure"][window_name]["Ivalue_blind"] = \
                    inputdata["WindowConfigure"][window_name]["windowIvalue"] / inputdata["WindowConfigure"][window_name]["glassIvalue"] \
                        * (-0.1331 * inputdata["WindowConfigure"][window_name]["glassIvalue"] ** 2 +\
                            0.8258 * inputdata["WindowConfigure"][window_name]["glassIvalue"] )

##----------------------------------------------------------------------------------
## 外壁の面積の計算（解説書 2.4.2.1）
##----------------------------------------------------------------------------------

# 外皮面積の算出
for room_zone_name in inputdata["EnvelopeSet"]:

    for wall_id, wall_configure in enumerate(inputdata["EnvelopeSet"][room_zone_name]["WallList"]):

        if inputdata["EnvelopeSet"][room_zone_name]["WallList"][wall_id]["EnvelopeArea"] == None:  # 外皮面積が空欄であれば、外皮の寸法から面積を計算。

            inputdata["EnvelopeSet"][room_zone_name]["WallList"][wall_id]["EnvelopeArea"] = \
                wall_configure["EnvelopeWidth"] * wall_configure["EnvelopeHeight"]

# 窓面積の算出
for window_id in inputdata["WindowConfigure"]:
    if inputdata["WindowConfigure"][window_id]["windowArea"] == None:  # 窓面積が空欄であれば、窓寸法から面積を計算。
        inputdata["WindowConfigure"][window_id]["windowArea"] = \
            inputdata["WindowConfigure"][window_id]["windowWidth"] * inputdata["WindowConfigure"][window_id]["windowHeight"]


# 外壁面積の算出
for room_zone_name in inputdata["EnvelopeSet"]:

    for (wall_id, wall_configure) in enumerate( inputdata["EnvelopeSet"][room_zone_name]["WallList"] ):

        window_total = 0  # 窓面積の集計用

        if "WindowList" in wall_configure:   # 窓がある場合

            # 窓面積の合計を求める（Σ{窓面積×枚数}）
            for (window_id, window_configure) in enumerate(wall_configure["WindowList"]):

                if window_configure["WindowID"] != "無":

                    window_total += \
                        inputdata["WindowConfigure"][ window_configure["WindowID"] ]["windowArea"] * window_configure["WindowNumber"]

        # 壁のみの面積（窓がない場合は、window_total = 0）
        if wall_configure["EnvelopeArea"] >= window_total:
            inputdata["EnvelopeSet"][room_zone_name]["WallList"][wall_id]["WallArea"] = wall_configure["EnvelopeArea"] - window_total
        else:
            print(room_zone_name)
            print(wall_configure)
            raise Exception('窓面積が外皮面積よりも大きくなっています')

##----------------------------------------------------------------------------------
## 室の定常熱取得の計算（解説書 2.4.2.2〜2.4.2.7）
##----------------------------------------------------------------------------------

## EnvelopeSet に WallConfigure, WindowConfigure の情報を貼り付ける。
for room_zone_name in inputdata["EnvelopeSet"]:

    # 壁毎にループ
    for (wall_id, wall_configure) in enumerate( inputdata["EnvelopeSet"][room_zone_name]["WallList"]):

        if inputdata["WallConfigure"][  wall_configure["WallSpec"]  ]["inputMethod"] == "断熱材種類を入力":

            if wall_configure["Direction"] == "水平（上）":  # 天井と見なす。

                # 外壁のUA（熱貫流率×面積）を計算
                inputdata["EnvelopeSet"][room_zone_name]["WallList"][wall_id]["UA_wall"] = \
                    inputdata["WallConfigure"][  wall_configure["WallSpec"]  ]["Uvalue_roof"] * wall_configure["WallArea"]

                # 動的負荷計算用
                inputdata["EnvelopeSet"][room_zone_name]["WallList"][wall_id]["U_wall"] = \
                    inputdata["WallConfigure"][  wall_configure["WallSpec"]  ]["Uvalue_roof"]
                inputdata["EnvelopeSet"][room_zone_name]["WallList"][wall_id]["WallArea"] = wall_configure["WallArea"]

            elif wall_configure["Direction"] == "水平（下）":  # 床と見なす。

                # 外壁のUA（熱貫流率×面積）を計算
                inputdata["EnvelopeSet"][room_zone_name]["WallList"][wall_id]["UA_wall"] = \
                    inputdata["WallConfigure"][  wall_configure["WallSpec"]  ]["Uvalue_floor"] * wall_configure["WallArea"]

                # 動的負荷計算用
                inputdata["EnvelopeSet"][room_zone_name]["WallList"][wall_id]["U_wall"] = \
                    inputdata["WallConfigure"][  wall_configure["WallSpec"]  ]["Uvalue_floor"]
                inputdata["EnvelopeSet"][room_zone_name]["WallList"][wall_id]["WallArea"] = wall_configure["WallArea"]

            else:

                # 外壁のUA（熱貫流率×面積）を計算
                inputdata["EnvelopeSet"][room_zone_name]["WallList"][wall_id]["UA_wall"] = \
                    inputdata["WallConfigure"][  wall_configure["WallSpec"]  ]["Uvalue_wall"] * wall_configure["WallArea"]

                # 動的負荷計算用
                inputdata["EnvelopeSet"][room_zone_name]["WallList"][wall_id]["U_wall"] = \
                    inputdata["WallConfigure"][  wall_configure["WallSpec"]  ]["Uvalue_wall"]
                inputdata["EnvelopeSet"][room_zone_name]["WallList"][wall_id]["WallArea"] = wall_configure["WallArea"]

        else:

            # 外壁のUA（熱貫流率×面積）を計算
            inputdata["EnvelopeSet"][room_zone_name]["WallList"][wall_id]["UA_wall"] = \
                inputdata["WallConfigure"][  wall_configure["WallSpec"]  ]["Uvalue"] * wall_configure["WallArea"]

            # 動的負荷計算用
            inputdata["EnvelopeSet"][room_zone_name]["WallList"][wall_id]["U_wall"] = \
                inputdata["WallConfigure"][  wall_configure["WallSpec"]  ]["Uvalue"]
            inputdata["EnvelopeSet"][room_zone_name]["WallList"][wall_id]["WallArea"] = wall_configure["WallArea"]

        for (window_id, window_configure) in enumerate( inputdata["EnvelopeSet"][room_zone_name]["WallList"][wall_id]["WindowList"]):

            if window_configure["WindowID"] != "無":

                # 日よけ効果係数の算出
                if window_configure["EavesID"] == "無":

                    inputdata["EnvelopeSet"][room_zone_name]["WallList"][wall_id]["WindowList"][window_id]["shadingEffect_C"] = 1
                    inputdata["EnvelopeSet"][room_zone_name]["WallList"][wall_id]["WindowList"][window_id]["shadingEffect_H"] = 1

                else:

                    if inputdata["ShadingConfigure"][ window_configure["EavesID"] ]["shadingEffect_C"] != None and \
                        inputdata["ShadingConfigure"][ window_configure["EavesID"] ]["shadingEffect_H"] != None :

                        inputdata["EnvelopeSet"][room_zone_name]["WallList"][wall_id]["WindowList"][window_id]["shadingEffect_C"] = \
                            inputdata["ShadingConfigure"][ window_configure["EavesID"] ]["shadingEffect_C"]
                        inputdata["EnvelopeSet"][room_zone_name]["WallList"][wall_id]["WindowList"][window_id]["shadingEffect_H"] = \
                            inputdata["ShadingConfigure"][ window_configure["EavesID"] ]["shadingEffect_H"]

                    else:

                        # 関数 shading.calc_shadingCoefficient で日よけ効果係数を算出。
                        (inputdata["EnvelopeSet"][room_zone_name]["WallList"][wall_id]["WindowList"][window_id]["shadingEffect_C"], \
                            inputdata["EnvelopeSet"][room_zone_name]["WallList"][wall_id]["WindowList"][window_id]["shadingEffect_H"] ) =  \
                                shading.calc_shadingCoefficient(inputdata["Building"]["Region"],\
                                    wall_configure["Direction"], \
                                    inputdata["ShadingConfigure"][ window_configure["EavesID"] ]["x1"],\
                                    inputdata["ShadingConfigure"][ window_configure["EavesID"] ]["x2"],\
                                    inputdata["ShadingConfigure"][ window_configure["EavesID"] ]["x3"],\
                                    inputdata["ShadingConfigure"][ window_configure["EavesID"] ]["y1"],\
                                    inputdata["ShadingConfigure"][ window_configure["EavesID"] ]["y2"],\
                                    inputdata["ShadingConfigure"][ window_configure["EavesID"] ]["y3"],\
                                    inputdata["ShadingConfigure"][ window_configure["EavesID"] ]["zxPlus"],\
                                    inputdata["ShadingConfigure"][ window_configure["EavesID"] ]["zxMinus"],\
                                    inputdata["ShadingConfigure"][ window_configure["EavesID"] ]["zyPlus"],\
                                    inputdata["ShadingConfigure"][ window_configure["EavesID"] ]["zyMinus"])


                # 窓のUA（熱貫流率×面積）を計算
                if window_configure["isBlind"] == "無":  # ブラインドがない場合

                    inputdata["EnvelopeSet"][room_zone_name]["WallList"][wall_id]["WindowList"][window_id]["UA_window"] = \
                        window_configure["WindowNumber"] * inputdata["WindowConfigure"][ window_configure["WindowID"] ]["windowArea"] * \
                        inputdata["WindowConfigure"][ window_configure["WindowID"] ]["Uvalue"]

                    inputdata["EnvelopeSet"][room_zone_name]["WallList"][wall_id]["WindowList"][window_id]["IA_window"] = \
                        window_configure["WindowNumber"] * inputdata["WindowConfigure"][ window_configure["WindowID"] ]["windowArea"] * \
                        inputdata["WindowConfigure"][ window_configure["WindowID"] ]["Ivalue"]

                    # 動的負荷計算用
                    inputdata["EnvelopeSet"][room_zone_name]["WallList"][wall_id]["WindowList"][window_id]["U_window"] = \
                        inputdata["WindowConfigure"][ window_configure["WindowID"] ]["Uvalue"]
                    inputdata["EnvelopeSet"][room_zone_name]["WallList"][wall_id]["WindowList"][window_id]["I_window"] = \
                        inputdata["WindowConfigure"][ window_configure["WindowID"] ]["Ivalue"]
                    inputdata["EnvelopeSet"][room_zone_name]["WallList"][wall_id]["WindowList"][window_id]["windowArea"] = \
                        window_configure["WindowNumber"] * inputdata["WindowConfigure"][ window_configure["WindowID"] ]["windowArea"]

                elif window_configure["isBlind"] == "有": # ブラインドがある場合

                    inputdata["EnvelopeSet"][room_zone_name]["WallList"][wall_id]["WindowList"][window_id]["UA_window"] = \
                        window_configure["WindowNumber"] * inputdata["WindowConfigure"][ window_configure["WindowID"] ]["windowArea"] * \
                        inputdata["WindowConfigure"][ window_configure["WindowID"] ]["Uvalue_blind"]

                    inputdata["EnvelopeSet"][room_zone_name]["WallList"][wall_id]["WindowList"][window_id]["IA_window"] = \
                        window_configure["WindowNumber"] * inputdata["WindowConfigure"][ window_configure["WindowID"] ]["windowArea"] * \
                        inputdata["WindowConfigure"][ window_configure["WindowID"] ]["Ivalue_blind"]

                    # 動的負荷計算用
                    inputdata["EnvelopeSet"][room_zone_name]["WallList"][wall_id]["WindowList"][window_id]["U_window"] = \
                        inputdata["WindowConfigure"][ window_configure["WindowID"] ]["Uvalue_blind"]
                    inputdata["EnvelopeSet"][room_zone_name]["WallList"][wall_id]["WindowList"][window_id]["I_window"] = \
                        inputdata["WindowConfigure"][ window_configure["WindowID"] ]["Ivalue_blind"]
                    inputdata["EnvelopeSet"][room_zone_name]["WallList"][wall_id]["WindowList"][window_id]["windowArea"] = \
                        window_configure["WindowNumber"] * inputdata["WindowConfigure"][ window_configure["WindowID"] ]["windowArea"]

                # 任意入力 SP-8
                if "window_Ivalue" in inputdata["SpecialInputData"]:
                    if window_configure["WindowID"] in inputdata["SpecialInputData"]["window_Ivalue"]:

                        inputdata["EnvelopeSet"][room_zone_name]["WallList"][wall_id]["WindowList"][window_id]["IA_window"] = \
                            window_configure["WindowNumber"] * inputdata["WindowConfigure"][ window_configure["WindowID"] ]["windowArea"] * \
                            np.array(inputdata["SpecialInputData"]["window_Ivalue"][ window_configure["WindowID"] ])


for room_zone_name in inputdata["AirConditioningZone"]:

    Qwall_T  = np.zeros(365)  # 壁からの温度差による熱取得 [W/m2]
    Qwall_S  = np.zeros(365)  # 壁からの日射による熱取得 [W/m2]
    Qwall_N  = np.zeros(365)  # 壁からの夜間放射による熱取得（マイナス）[W/m2]
    Qwind_T  = np.zeros(365)  # 窓からの温度差による熱取得 [W/m2]
    Qwind_S  = np.zeros(365)  # 窓からの日射による熱取得 [W/m2]
    Qwind_N  = np.zeros(365)  # 窓からの夜間放射による熱取得（マイナス）[W/m2]

    # 外壁があれば以下を実行
    if room_zone_name in inputdata["EnvelopeSet"]:

        # 壁毎にループ
        for (wall_id, wall_configure) in enumerate( inputdata["EnvelopeSet"][room_zone_name]["WallList"]):

            if wall_configure["WallType"] == "日の当たる外壁":

                ## ① 温度差による熱取得
                Qwall_T = Qwall_T + wall_configure["UA_wall"] * (Toa_ave - TroomSP) * 24

                ## ② 日射による熱取得
                if wall_configure["Direction"] == "水平（上）" or wall_configure["Direction"] == "水平（下）":
                    Qwall_S = Qwall_S + wall_configure["UA_wall"] * 0.8 * 0.04 * \
                        (solor_radiation["直達"]["水平"]+solor_radiation["天空"]["水平"])
                else:
                    Qwall_S = Qwall_S + wall_configure["UA_wall"] * 0.8 * 0.04 * \
                        (solor_radiation["直達"][ wall_configure["Direction"] ]+solor_radiation["天空"]["垂直"])

                ## ③ 夜間放射による熱取得（マイナス）
                if wall_configure["Direction"] == "水平（上）" or wall_configure["Direction"] == "水平（下）":
                    Qwall_N = Qwall_N - wall_configure["UA_wall"] * 0.9 * 0.04 * \
                        (solor_radiation["夜間"]["水平"])
                else:
                    Qwall_N = Qwall_N - wall_configure["UA_wall"] * 0.9 * 0.04 * \
                        (solor_radiation["夜間"]["垂直"])

            elif wall_configure["WallType"] == "日の当たらない外壁":

                ## ① 温度差による熱取得
                Qwall_T = Qwall_T + wall_configure["UA_wall"] * (Toa_ave - TroomSP) * 24

                ## ③ 夜間放射による熱取得（マイナス）
                if wall_configure["Direction"] == "水平（上）" or wall_configure["Direction"] == "水平（下）":
                    Qwall_N = Qwall_N - wall_configure["UA_wall"] * 0.9 * 0.04 * \
                        (solor_radiation["夜間"]["水平"])
                else:
                    Qwall_N = Qwall_N - wall_configure["UA_wall"] * 0.9 * 0.04 * \
                        (solor_radiation["夜間"]["垂直"])

            elif wall_configure["WallType"] == "地盤に接する外壁":

                ## ① 温度差による熱取得
                Qwall_T = Qwall_T + wall_configure["UA_wall"] * (np.mean(Toa_ave)* np.ones(365) - TroomSP) * 24

                ## ③ 夜間放射による熱取得（マイナス） ：　本当はこれは不要。Webproの実装と合わせるために追加。
                Qwall_N = Qwall_N - wall_configure["UA_wall"] * 0.9 * 0.04 * (solor_radiation["夜間"]["垂直"])

            elif wall_configure["WallType"] == "地盤に接する外壁_Ver2":  # Webpro Ver2の互換のための処理

                ## ① 温度差による熱取得
                Qwall_T = Qwall_T + wall_configure["UA_wall"] * (np.mean(Toa_ave)* np.ones(365) - TroomSP) * 24

                ## ② 日射による熱取得
                if wall_configure["Direction"] == "水平（上）" or wall_configure["Direction"] == "水平（下）":
                    Qwall_S = Qwall_S + wall_configure["UA_wall"] * 0.8 * 0.04 * \
                        (solor_radiation["直達"]["水平"]+solor_radiation["天空"]["水平"])
                else:
                    Qwall_S = Qwall_S + wall_configure["UA_wall"] * 0.8 * 0.04 * \
                        (solor_radiation["直達"][ wall_configure["Direction"] ]+solor_radiation["天空"]["垂直"])

                ## ③ 夜間放射による熱取得（マイナス）
                if wall_configure["Direction"] == "水平（上）" or wall_configure["Direction"] == "水平（下）":
                    Qwall_N = Qwall_N - wall_configure["UA_wall"] * 0.9 * 0.04 * \
                        (solor_radiation["夜間"]["水平"])
                else:
                    Qwall_N = Qwall_N - wall_configure["UA_wall"] * 0.9 * 0.04 * \
                        (solor_radiation["夜間"]["垂直"])


            # 窓毎にループ
            for (window_id, window_configure) in enumerate( wall_configure["WindowList"]):

                if window_configure["WindowID"] != "無":  # 窓がある場合

                    if wall_configure["WallType"] == "日の当たる外壁" or wall_configure["WallType"] == "地盤に接する外壁_Ver2":

                        ## ① 温度差による熱取得
                        Qwind_T = Qwind_T + window_configure["UA_window"]*(Toa_ave-TroomSP)*24

                        ## ② 日射による熱取得
                        shading_daily = np.zeros(365)

                        for dd in range(0,365):

                            if ac_mode[dd] == "冷房":
                                shading_daily[dd] = window_configure["shadingEffect_C"]
                            elif ac_mode[dd] == "中間":
                                shading_daily[dd] = window_configure["shadingEffect_C"]
                            elif ac_mode[dd] == "暖房":
                                shading_daily[dd] = window_configure["shadingEffect_H"]

                        if isinstance(window_configure["IA_window"], float):

                            # 様式2-3に入力された窓仕様を使用する場合
                            # 0.88は標準ガラスの日射熱取得率
                            # 0.89は標準ガラスの入射角特性の最大値
                            # 0.808は天空・反射日射に対する標準ガラスの入射角特性 0.808/0.88 = 0.91818
                            if wall_configure["Direction"] == "水平（上）" or wall_configure["Direction"] == "水平（下）":

                                Qwind_S = Qwind_S + shading_daily * \
                                    (window_configure["IA_window"] / 0.88) * \
                                    (solor_radiation["直達_入射角特性込"]["水平"]*0.89 + solor_radiation["天空"]["水平"]*0.808)

                            else:

                                Qwind_S = Qwind_S + shading_daily * \
                                    (window_configure["IA_window"] / 0.88) * \
                                    (solor_radiation["直達_入射角特性込"][ wall_configure["Direction"] ]*0.89 + solor_radiation["天空"]["垂直"]*0.808)

                        else:

                            # 任意入力の場合（SP-8）
                            if wall_configure["Direction"] == "水平（上）" or wall_configure["Direction"] == "水平（下）":
                                Qwind_S = Qwind_S +   \
                                    (window_configure["IA_window"]) * (solor_radiation["直達"]["水平"] + solor_radiation["天空"]["水平"])
                            else:
                                Qwind_S = Qwind_S + shading_daily * \
                                    (window_configure["IA_window"]) * (solor_radiation["直達"][ wall_configure["Direction"] ] + solor_radiation["天空"]["垂直"])


                        ## ③ 夜間放射による熱取得（マイナス）
                        if wall_configure["Direction"] == "水平（上）" or wall_configure["Direction"] == "水平（下）":
                            Qwind_N = Qwind_N - window_configure["UA_window"] * 0.9 * 0.04 * solor_radiation["夜間"]["水平"]
                        else:
                            Qwind_N = Qwind_N - window_configure["UA_window"] * 0.9 * 0.04 * solor_radiation["夜間"]["垂直"]


                    elif wall_configure["WallType"] == "日の当たらない外壁":

                        ## ③ 夜間放射による熱取得（マイナス）
                        Qwind_N = Qwind_N - window_configure["UA_window"] * 0.9 * 0.04 * solor_radiation["夜間"]["水平"]




    #  室面積あたりの熱量に変換 [Wh/m2/日]
    resultJson["Qroom"][room_zone_name]["Qwall_T"] = Qwall_T / inputdata["AirConditioningZone"][room_zone_name]["zoneArea"]
    resultJson["Qroom"][room_zone_name]["Qwall_S"] = Qwall_S / inputdata["AirConditioningZone"][room_zone_name]["zoneArea"]
    resultJson["Qroom"][room_zone_name]["Qwall_N"] = Qwall_N / inputdata["AirConditioningZone"][room_zone_name]["zoneArea"]
    resultJson["Qroom"][room_zone_name]["Qwind_T"] = Qwind_T / inputdata["AirConditioningZone"][room_zone_name]["zoneArea"]
    resultJson["Qroom"][room_zone_name]["Qwind_S"] = Qwind_S / inputdata["AirConditioningZone"][room_zone_name]["zoneArea"]
    resultJson["Qroom"][room_zone_name]["Qwind_N"] = Qwind_N / inputdata["AirConditioningZone"][room_zone_name]["zoneArea"]



##----------------------------------------------------------------------------------
## 室負荷の計算（解説書 2.4.3、2.4.4）
##----------------------------------------------------------------------------------

## 室負荷計算のための係数（解説書 A.3）
with open(database_directory + 'QROOM_COEFFI_AREA'+ inputdata["Building"]["Region"] +'.json', 'r', encoding='utf-8') as f:
    QROOM_COEFFI = json.load(f)


Heat_light_hourly = {}
Num_of_Person_hourly = {}
Heat_OAapp_hourly = {}

for room_zone_name in inputdata["AirConditioningZone"]:

    Qroom_CTC = np.zeros(365)
    Qroom_CTH = np.zeros(365)
    Qroom_CSR = np.zeros(365)

    Qcool     = np.zeros(365)
    Qheat     = np.zeros(365)

    # 室が使用されているか否か＝空調運転時間（365日分）
    room_usage = np.sum(roomScheduleRoom[room_zone_name],1)

    btype = inputdata["AirConditioningZone"][room_zone_name]["buildingType"]
    rtype = inputdata["AirConditioningZone"][room_zone_name]["roomType"]

    # 発熱量参照値 [W/m2] を読み込む関数（空調）
    if "room_usage_condition" in inputdata["SpecialInputData"]:
        (roomHeatGain_Light, roomHeatGain_Person, roomHeatGain_OAapp, roomNumOfPerson) = \
            bc.get_roomHeatGain(btype, rtype, inputdata["SpecialInputData"]["room_usage_condition"])
    else:
        (roomHeatGain_Light, roomHeatGain_Person, roomHeatGain_OAapp, roomNumOfPerson) = \
            bc.get_roomHeatGain(btype, rtype)

    Heat_light_daily  = np.sum(roomScheduleLight[room_zone_name],1) * roomHeatGain_Light   # 照明からの発熱（日積算）（365日分）
    Heat_person_daily = np.sum(roomSchedulePerson[room_zone_name],1) * roomHeatGain_Person # 人体からの発熱（日積算）（365日分）
    Heat_OAapp_daily  = np.sum(roomScheduleOAapp[room_zone_name],1) * roomHeatGain_OAapp   # 機器からの発熱（日積算）（365日分）

    # 時刻別計算用（本来はこのループに入れるべきではない → 時刻別計算の方に入れるべき）
    Heat_light_hourly[room_zone_name]  = roomScheduleLight[room_zone_name] * roomHeatGain_Light   # 照明からの発熱 （365日分）
    Num_of_Person_hourly[room_zone_name] = roomSchedulePerson[room_zone_name] * roomNumOfPerson   # 人員密度（365日分）
    Heat_OAapp_hourly[room_zone_name]  = roomScheduleOAapp[room_zone_name] * roomHeatGain_OAapp   # 機器からの発熱 （365日分）


    for dd in range(0,365):

        if room_usage[dd] > 0:

            # 前日の空調の有無
            if "終日空調" in QROOM_COEFFI[ btype ][ rtype ]:
                onoff = "終日空調"
            elif (dd > 0) and (room_usage[dd-1] > 0):
                onoff = "前日空調"
            else:
                onoff = "前日休み"

            if ac_mode[dd] == "冷房":

                Qroom_CTC[dd] = QROOM_COEFFI[ btype ][ rtype ][onoff]["冷房期"]["外気温変動"]["冷房負荷"]["係数"] * \
                    ( resultJson["Qroom"][room_zone_name]["Qwall_T"][dd] + resultJson["Qroom"][room_zone_name]["Qwall_N"][dd] + \
                    resultJson["Qroom"][room_zone_name]["Qwind_T"][dd] + resultJson["Qroom"][room_zone_name]["Qwind_N"][dd] ) + \
                    QROOM_COEFFI[ btype ][ rtype ][onoff]["冷房期"]["外気温変動"]["冷房負荷"]["補正切片"]

                Qroom_CTH[dd] = QROOM_COEFFI[ btype ][ rtype ][onoff]["冷房期"]["外気温変動"]["暖房負荷"]["係数"] * \
                    ( resultJson["Qroom"][room_zone_name]["Qwall_T"][dd] + resultJson["Qroom"][room_zone_name]["Qwall_N"][dd] + \
                    resultJson["Qroom"][room_zone_name]["Qwind_T"][dd] + resultJson["Qroom"][room_zone_name]["Qwind_N"][dd] ) + \
                    QROOM_COEFFI[ btype ][ rtype ][onoff]["冷房期"]["外気温変動"]["暖房負荷"]["補正切片"]

                Qroom_CSR[dd] = QROOM_COEFFI[ btype ][ rtype ][onoff]["冷房期"]["日射量変動"]["冷房負荷"]["係数"] * \
                    ( resultJson["Qroom"][room_zone_name]["Qwall_S"][dd] + resultJson["Qroom"][room_zone_name]["Qwind_S"][dd] ) + \
                    QROOM_COEFFI[ btype ][ rtype ][onoff]["冷房期"]["日射量変動"]["冷房負荷"]["切片"]

            elif ac_mode[dd] == "暖房":

                Qroom_CTC[dd] = QROOM_COEFFI[ btype ][ rtype ][onoff]["暖房期"]["外気温変動"]["冷房負荷"]["係数"] * \
                    ( resultJson["Qroom"][room_zone_name]["Qwall_T"][dd] + resultJson["Qroom"][room_zone_name]["Qwall_N"][dd] + \
                    resultJson["Qroom"][room_zone_name]["Qwind_T"][dd] + resultJson["Qroom"][room_zone_name]["Qwind_N"][dd] ) + \
                    QROOM_COEFFI[ btype ][ rtype ][onoff]["暖房期"]["外気温変動"]["冷房負荷"]["切片"]

                Qroom_CTH[dd] = QROOM_COEFFI[ btype ][ rtype ][onoff]["暖房期"]["外気温変動"]["暖房負荷"]["係数"] * \
                    ( resultJson["Qroom"][room_zone_name]["Qwall_T"][dd] + resultJson["Qroom"][room_zone_name]["Qwall_N"][dd] + \
                    resultJson["Qroom"][room_zone_name]["Qwind_T"][dd] + resultJson["Qroom"][room_zone_name]["Qwind_N"][dd] ) + \
                    QROOM_COEFFI[ btype ][ rtype ][onoff]["暖房期"]["外気温変動"]["暖房負荷"]["切片"]

                Qroom_CSR[dd] = QROOM_COEFFI[ btype ][ rtype ][onoff]["暖房期"]["日射量変動"]["冷房負荷"]["係数"] * \
                    ( resultJson["Qroom"][room_zone_name]["Qwall_S"][dd] + resultJson["Qroom"][room_zone_name]["Qwind_S"][dd] ) + \
                    QROOM_COEFFI[ btype ][ rtype ][onoff]["暖房期"]["日射量変動"]["冷房負荷"]["切片"]

            elif ac_mode[dd] == "中間":

                Qroom_CTC[dd] = QROOM_COEFFI[ btype ][ rtype ][onoff]["中間期"]["外気温変動"]["冷房負荷"]["係数"] * \
                    ( resultJson["Qroom"][room_zone_name]["Qwall_T"][dd] + resultJson["Qroom"][room_zone_name]["Qwall_N"][dd] + \
                    resultJson["Qroom"][room_zone_name]["Qwind_T"][dd] + resultJson["Qroom"][room_zone_name]["Qwind_N"][dd] ) + \
                    QROOM_COEFFI[ btype ][ rtype ][onoff]["中間期"]["外気温変動"]["冷房負荷"]["補正切片"]

                Qroom_CTH[dd] = QROOM_COEFFI[ btype ][ rtype ][onoff]["中間期"]["外気温変動"]["暖房負荷"]["係数"] * \
                    ( resultJson["Qroom"][room_zone_name]["Qwall_T"][dd] + resultJson["Qroom"][room_zone_name]["Qwall_N"][dd] + \
                    resultJson["Qroom"][room_zone_name]["Qwind_T"][dd] + resultJson["Qroom"][room_zone_name]["Qwind_N"][dd] ) + \
                    QROOM_COEFFI[ btype ][ rtype ][onoff]["中間期"]["外気温変動"]["暖房負荷"]["補正切片"]

                Qroom_CSR[dd] = QROOM_COEFFI[ btype ][ rtype ][onoff]["中間期"]["日射量変動"]["冷房負荷"]["係数"] * \
                    ( resultJson["Qroom"][room_zone_name]["Qwall_S"][dd] + resultJson["Qroom"][room_zone_name]["Qwind_S"][dd] ) + \
                    QROOM_COEFFI[ btype ][ rtype ][onoff]["中間期"]["日射量変動"]["冷房負荷"]["切片"]

            if Qroom_CTC[dd] < 0:
                Qroom_CTC[dd] = 0

            if Qroom_CTH[dd] > 0:
                Qroom_CTH[dd] = 0

            if Qroom_CSR[dd] < 0:
                Qroom_CSR[dd] = 0

            # 日射負荷 Qroom_CSR を暖房負荷 Qroom_CTH に足す
            Qcool[dd] = Qroom_CTC[dd]
            Qheat[dd] = Qroom_CTH[dd] + Qroom_CSR[dd]

            # 日射負荷によって暖房負荷がプラスになった場合は、超過分を冷房負荷に加算
            if Qheat[dd] > 0:
                Qcool[dd] = Qcool[dd] + Qheat[dd]
                Qheat[dd] = 0

            # 内部発熱を暖房負荷 Qheat に足す
            Qheat[dd] = Qheat[dd] + ( Heat_light_daily[dd] + Heat_person_daily[dd] + Heat_OAapp_daily[dd] )

            # 内部発熱によって暖房負荷がプラスになった場合は、超過分を冷房負荷に加算
            if Qheat[dd] > 0:
                Qcool[dd] = Qcool[dd] + Qheat[dd]
                Qheat[dd] = 0

        else:

            # 空調OFF時は 0 とする
            Qcool[dd] = 0
            Qheat[dd] = 0


    # 日積算熱取得　　QroomDc, QroomDh [MJ/day]
    resultJson["Qroom"][room_zone_name]["QroomDc"] = Qcool * (3600/1000000) * inputdata["AirConditioningZone"][room_zone_name]["zoneArea"]
    resultJson["Qroom"][room_zone_name]["QroomDh"] = Qheat * (3600/1000000) * inputdata["AirConditioningZone"][room_zone_name]["zoneArea"]


In [72]:
#実行
resultdata_AC = {}

if exec_calculation:

    try:
        if inputdata["AirConditioningZone"]:   # AirConditioningZone が 空 でなければ

            resultdata_AC = airconditioning_webpro.calc_energy(inputdata, debug = True)

            # CGSの計算に必要となる変数
            resultJson_for_CGS["AC"] = resultdata_AC["for_CGS"]

            # 設計一次エネ・基準一次エネに追加
            energy_consumption_design += resultdata_AC["設計一次エネルギー消費量[MJ/年]"]
            energy_consumption_standard += resultdata_AC["基準一次エネルギー消費量[MJ/年]"]
            calc_reuslt["設計一次エネルギー消費量（空調）[MJ]"] = resultdata_AC["設計一次エネルギー消費量[MJ/年]"]
            calc_reuslt["基準一次エネルギー消費量（空調）[MJ]"] = resultdata_AC["基準一次エネルギー消費量[MJ/年]"]
            calc_reuslt["BEI_AC"] = math.ceil(resultdata_AC["BEI/AC"] * 100) / 100

        else:
            resultdata_AC = {
                "message": "空気調和設備はありません。"
            }

    except:
        resultdata_AC = {
            "error": "空気調和設備の計算時に予期せぬエラーが発生しました。"
        }

else:
    resultdata_AC = {
        "error": "空気調和設備の計算は実行されませんでした。"
    }

# 出力
with open(inputfile_name_split[0] + "_result_AC.json",'w', encoding='utf-8') as fw:
    json.dump(resultdata_AC, fw, indent=4, ensure_ascii=False, cls = MyEncoder)


--- ゾーン名 1F_ロビー ---
熱取得_壁温度 Qwall_T: 0.0
熱取得_壁日射 Qwall_S: 0.0
熱取得_壁放射 Qwall_N: 0.0
熱取得_窓温度 Qwind_T: 0.0
熱取得_窓日射 Qwind_S: 0.0
熱取得_窓放射 Qwind_N: 0.0
--- ゾーン名 1F_EVホール ---
熱取得_壁温度 Qwall_T: 0.0
熱取得_壁日射 Qwall_S: 0.0
熱取得_壁放射 Qwall_N: 0.0
熱取得_窓温度 Qwind_T: 0.0
熱取得_窓日射 Qwind_S: 0.0
熱取得_窓放射 Qwind_N: 0.0
--- ゾーン名 1F_中央監視室・警備室 ---
熱取得_壁温度 Qwall_T: 0.0
熱取得_壁日射 Qwall_S: 0.0
熱取得_壁放射 Qwall_N: 0.0
熱取得_窓温度 Qwind_T: 0.0
熱取得_窓日射 Qwind_S: 0.0
熱取得_窓放射 Qwind_N: 0.0
--- ゾーン名 1F_更衣室1 ---
熱取得_壁温度 Qwall_T: 0.0
熱取得_壁日射 Qwall_S: 0.0
熱取得_壁放射 Qwall_N: 0.0
熱取得_窓温度 Qwind_T: 0.0
熱取得_窓日射 Qwind_S: 0.0
熱取得_窓放射 Qwind_N: 0.0
--- ゾーン名 1F_更衣室2 ---
熱取得_壁温度 Qwall_T: 0.0
熱取得_壁日射 Qwall_S: 0.0
熱取得_壁放射 Qwall_N: 0.0
熱取得_窓温度 Qwind_T: 0.0
熱取得_窓日射 Qwind_S: 0.0
熱取得_窓放射 Qwind_N: 0.0
--- ゾーン名 1F_休憩室 ---
熱取得_壁温度 Qwall_T: 0.0
熱取得_壁日射 Qwall_S: 0.0
熱取得_壁放射 Qwall_N: 0.0
熱取得_窓温度 Qwind_T: 0.0
熱取得_窓日射 Qwind_S: 0.0
熱取得_窓放射 Qwind_N: 0.0
--- ゾーン名 1F_事務室1 ---
熱取得_壁温度 Qwall_T: 0.0
熱取得_壁日射 Qwall_S: 0.0
熱取得_壁放射 Qwall_N: 0.0
熱取得_窓温度 Qwind_T: 0.0
熱取得_窓日射 Q

In [73]:
enumerate(inputdata["EnvelopeSet"][room_zone_name]["WallList"])

In [74]:
for room_zone_name in inputdata["EnvelopeSet"]:
  for (wall_id, wall_configure) in enumerate( inputdata["EnvelopeSet"][room_zone_name]["WallList"]):
    print(wall_id, wall_configure)

0 {'Direction': '南', 'EnvelopeArea': 50.0, 'EnvelopeWidth': None, 'EnvelopeHeight': None, 'WallSpec': 'W1', 'WallType': '日の当たる外壁', 'WindowList': [{'WindowID': 'G1', 'WindowNumber': 16.64, 'isBlind': '無', 'EavesID': '無', 'Info': None, 'shadingEffect_C': 1, 'shadingEffect_H': 1, 'UA_window': 94.3488, 'IA_window': 11.648, 'U_window': 5.67, 'I_window': 0.7, 'windowArea': 16.64}], 'WallArea': 33.36, 'UA_wall': 32.72280162225658, 'U_wall': 0.9808993292043341}
1 {'Direction': '北', 'EnvelopeArea': 114.12, 'EnvelopeWidth': None, 'EnvelopeHeight': None, 'WallSpec': 'FG1', 'WallType': '地盤に接する外壁', 'WindowList': [{'WindowID': '無', 'WindowNumber': None, 'isBlind': '無', 'EavesID': '無', 'Info': None}], 'WallArea': 114.12, 'UA_wall': 411.18475323567066, 'U_wall': 3.6030910728677763}
0 {'Direction': '北', 'EnvelopeArea': 16.32, 'EnvelopeWidth': None, 'EnvelopeHeight': None, 'WallSpec': 'FG1', 'WallType': '地盤に接する外壁', 'WindowList': [{'WindowID': '無', 'WindowNumber': None, 'isBlind': '無', 'EavesID': '無', 'I

In [75]:
inputdata["EnvelopeSet"][room_zone_name]["WallList"][0]["U_wall"]
inputdata["AirHandlingSystem"]

{'EHP1-1': {'isEconomizer': '無',
  'EconomizerMaxAirVolume': None,
  'isOutdoorAirCut': '無',
  'Pump_cooling': 'dummyPump_0',
  'Pump_heating': 'dummyPump_1',
  'HeatSource_cooling': 'EHP1-1O',
  'HeatSource_heating': 'EHP1-1O',
  'AirHandlingUnit': [{'Type': '室内機',
    'Number': 1.0,
    'RatedCapacityCooling': 5.6,
    'RatedCapacityHeating': 6.3,
    'FanType': None,
    'FanAirVolume': None,
    'FanPowerConsumption': 0.05,
    'FanControlType': '定風量制御',
    'FanMinOpeningRate': None,
    'AirHeatExchangeRatioCooling': None,
    'AirHeatExchangeRatioHeating': None,
    'AirHeatExchangerEffectiveAirVolumeRatio': None,
    'AirHeatExchangerControl': '無',
    'AirHeatExchangerPowerConsumption': None,
    'Info': 'EHP1-1O',
    'energy_consumption_ratio': array([1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1.2]),
    'FanPowerConsumption_total': 0.05},
   {'Type': '全熱交ユニット',
    'Number': 1.0,
    'RatedCapacityCooling': None,
    'RatedCapacityHeating': None,
    'FanType': None,

In [76]:

    for ahu_name in inputdata["AirHandlingSystem"]:

        resultJson["AHU"][ahu_name] = {

            "day_mode": [],                    # 空調機群の運転時間帯（昼、夜、終日）
            "schedule": np.zeros((365,24)),    # 時刻別の運転スケジュール（365×24）
            "HoaDayAve": np.zeros(365),        # 空調運転時間帯の外気エンタルピー

            "qoaAHU": np.zeros(365),           # 日平均外気負荷 [kW]
            "Tahu_total": np.zeros(365),       # 空調機群の日積算運転時間（冷暖合計）

            "E_fan_day": np.zeros(365),        # 空調機群のエネルギー消費量
            "E_fan_c_day": np.zeros(365),      # 空調機群のエネルギー消費量（冷房）
            "E_fan_h_day": np.zeros(365),      # 空調機群のエネルギー消費量（暖房）
            "E_AHUaex_day": np.zeros(365),     # 全熱交換器のエネルギー消費量

            "TdAHUc_total": np.zeros(365),     # 空調機群の冷房運転時間の合計
            "TdAHUh_total": np.zeros(365),     # 空調機群の暖房運転時間の合計

            "Qahu_remainC": np.zeros(365),     # 空調機群の未処理負荷（冷房）[MJ/day]
            "Qahu_remainH": np.zeros(365),     # 空調機群の未処理負荷（暖房）[MJ/day]

            "energy_consumption_each_LF": np.zeros(len(aveL)),

            "Qroom" :{
                "cooling_for_room" : np.zeros(365),   # 日積算室負荷（冷房要求）の積算値 [MJ/day]
                "heating_for_room" : np.zeros(365),   # 日積算室負荷（暖房要求）の積算値 [MJ/day]
            },
            "Qahu" :{
                "cooling_for_room" : np.zeros(365),   # 室負荷が正（冷房要求）であるときの空調負荷（正であれば冷却コイル負荷、負であれば加熱コイル負荷） [MJ/day]
                "heating_for_room" : np.zeros(365),   # 室負荷が負（暖房要求）であるときの空調負荷（正であれば冷却コイル負荷、負であれば加熱コイル負荷） [MJ/day]
            },
            "Tahu" :{
                "cooling_for_room" : np.zeros(365),   # 室負荷が冷房要求である場合の空調機群の運転時間 [h/day]
                "heating_for_room" : np.zeros(365),   # 室負荷が暖房要求である場合の空調機群の運転時間 [h/day]
            },

            "Economizer":{
                "AHUVovc" : np.zeros(365),   # 外気冷房運転時の外気風量 [kg/s]
                "Qahu_oac": np.zeros(365),   # 外気冷房による負荷削減効果 [MJ/day]
            },

            "LdAHUc": {
                "cooling_for_room" : np.zeros(365), # 空調機群の冷房運転時の負荷率帯（室負荷が冷房要求である場合）
                "heating_for_room" : np.zeros(365), # 空調機群の冷房運転時の負荷率帯（室負荷が暖房要求である場合）
            },
            "TdAHUc": {
                "cooling_for_room" : np.zeros(365), # 空調機群の冷房運転時間（室負荷が冷房要求である場合）
                "heating_for_room" : np.zeros(365), # 空調機群の冷房運転時間（室負荷が冷房要求である場合）
            },
            "LdAHUh": {
                "cooling_for_room" : np.zeros(365), # 空調機群の暖房負荷率帯（室負荷が冷房要求である場合）
                "heating_for_room" : np.zeros(365), # 空調機群の暖房負荷率帯（室負荷が冷房要求である場合）
            },
            "TdAHUh": {
                "cooling_for_room" : np.zeros(365), # 空調機群の暖房運転時間（室負荷が冷房要求である場合）
                "heating_for_room" : np.zeros(365), # 空調機群の暖房運転時間（室負荷が冷房要求である場合）
            },

            "TcAHU": 0,
            "ThAHU": 0,
            "MxAHUcE": 0,
            "MxAHUhE": 0
        }


In [77]:

for ahu_name in inputdata["AirHandlingSystem"]:

    # 空調機タイプ（1つでも空調機があれば「空調機」と判断する）
    inputdata["AirHandlingSystem"][ahu_name]["AHU_type"] = "空調機以外"
    for unit_id, unit_configure in enumerate(inputdata["AirHandlingSystem"][ahu_name]["AirHandlingUnit"]):
        if unit_configure["Type"] == "空調機":
            inputdata["AirHandlingSystem"][ahu_name]["AHU_type"] = "空調機"
            break

    # 空調機の能力
    inputdata["AirHandlingSystem"][ahu_name]["RatedCapacityCooling"] = 0
    inputdata["AirHandlingSystem"][ahu_name]["RatedCapacityHeating"] = 0
    for unit_id, unit_configure in enumerate(inputdata["AirHandlingSystem"][ahu_name]["AirHandlingUnit"]):
        if unit_configure["RatedCapacityCooling"] != None:
            inputdata["AirHandlingSystem"][ahu_name]["RatedCapacityCooling"] += \
                unit_configure["RatedCapacityCooling"] * unit_configure["Number"]

        if unit_configure["RatedCapacityHeating"] != None:
            inputdata["AirHandlingSystem"][ahu_name]["RatedCapacityHeating"] += \
                unit_configure["RatedCapacityHeating"] * unit_configure["Number"]

    # 空調機の風量 [m3/h]
    inputdata["AirHandlingSystem"][ahu_name]["FanAirVolume"] = 0
    for unit_id, unit_configure in enumerate(inputdata["AirHandlingSystem"][ahu_name]["AirHandlingUnit"]):
        if unit_configure["FanAirVolume"] != None:
            inputdata["AirHandlingSystem"][ahu_name]["FanAirVolume"] += \
                unit_configure["FanAirVolume"] * unit_configure["Number"]

    # 全熱交換器の効率（一番低いものを採用）
    inputdata["AirHandlingSystem"][ahu_name]["AirHeatExchangeRatioCooling"] = None
    inputdata["AirHandlingSystem"][ahu_name]["AirHeatExchangeRatioHeating"] = None
    for unit_id, unit_configure in enumerate(inputdata["AirHandlingSystem"][ahu_name]["AirHandlingUnit"]):

        # 冷房の効率
        if (unit_configure["AirHeatExchangeRatioCooling"] != None):
            if inputdata["AirHandlingSystem"][ahu_name]["AirHeatExchangeRatioCooling"] == None:
                inputdata["AirHandlingSystem"][ahu_name]["AirHeatExchangeRatioCooling"] = unit_configure["AirHeatExchangeRatioCooling"]
            elif inputdata["AirHandlingSystem"][ahu_name]["AirHeatExchangeRatioCooling"] > unit_configure["AirHeatExchangeRatioCooling"]:
                inputdata["AirHandlingSystem"][ahu_name]["AirHeatExchangeRatioCooling"] = unit_configure["AirHeatExchangeRatioCooling"]

        # 暖房の効率
        if (unit_configure["AirHeatExchangeRatioHeating"] != None):
            if inputdata["AirHandlingSystem"][ahu_name]["AirHeatExchangeRatioHeating"] == None:
                inputdata["AirHandlingSystem"][ahu_name]["AirHeatExchangeRatioHeating"] = unit_configure["AirHeatExchangeRatioHeating"]
            elif inputdata["AirHandlingSystem"][ahu_name]["AirHeatExchangeRatioHeating"] > unit_configure["AirHeatExchangeRatioHeating"]:
                inputdata["AirHandlingSystem"][ahu_name]["AirHeatExchangeRatioHeating"] = unit_configure["AirHeatExchangeRatioHeating"]

    # 全熱交換器のバイパス制御の有無（1つでもあればバイパス制御「有」とする）
    inputdata["AirHandlingSystem"][ahu_name]["AirHeatExchangerControl"] = "無"
    for unit_id, unit_configure in enumerate(inputdata["AirHandlingSystem"][ahu_name]["AirHandlingUnit"]):
        if (unit_configure["AirHeatExchangeRatioCooling"] != None) and (unit_configure["AirHeatExchangeRatioHeating"] != None):
            if unit_configure["AirHeatExchangerControl"] == "有":
                inputdata["AirHandlingSystem"][ahu_name]["AirHeatExchangerControl"] = "有"

    # 全熱交換器の消費電力 [kW]
    inputdata["AirHandlingSystem"][ahu_name]["AirHeatExchangerPowerConsumption"] = 0
    for unit_id, unit_configure in enumerate(inputdata["AirHandlingSystem"][ahu_name]["AirHandlingUnit"]):
        if unit_configure["AirHeatExchangerPowerConsumption"] != None:
            inputdata["AirHandlingSystem"][ahu_name]["AirHeatExchangerPowerConsumption"] += \
                unit_configure["AirHeatExchangerPowerConsumption"] * unit_configure["Number"]

    # 全熱交換器の風量 [m3/h]
    inputdata["AirHandlingSystem"][ahu_name]["AirHeatExchangerAirVolume"] = 0
    for unit_id, unit_configure in enumerate(inputdata["AirHandlingSystem"][ahu_name]["AirHandlingUnit"]):
        if (unit_configure["AirHeatExchangeRatioCooling"] != None) and (unit_configure["AirHeatExchangeRatioHeating"] != None):
            inputdata["AirHandlingSystem"][ahu_name]["AirHeatExchangerAirVolume"]  += \
                unit_configure["FanAirVolume"] * unit_configure["Number"]


In [78]:
inputdata["AirHandlingSystem"]['EHP1-1']
for unit_id, unit_configure in enumerate(inputdata["AirHandlingSystem"][ahu_name]["AirHandlingUnit"]):
  print(unit_id)

0


In [79]:
inputdata["AirHandlingSystem"]

{'EHP1-1': {'isEconomizer': '無',
  'EconomizerMaxAirVolume': None,
  'isOutdoorAirCut': '無',
  'Pump_cooling': 'dummyPump_0',
  'Pump_heating': 'dummyPump_1',
  'HeatSource_cooling': 'EHP1-1O',
  'HeatSource_heating': 'EHP1-1O',
  'AirHandlingUnit': [{'Type': '室内機',
    'Number': 1.0,
    'RatedCapacityCooling': 5.6,
    'RatedCapacityHeating': 6.3,
    'FanType': None,
    'FanAirVolume': None,
    'FanPowerConsumption': 0.05,
    'FanControlType': '定風量制御',
    'FanMinOpeningRate': None,
    'AirHeatExchangeRatioCooling': None,
    'AirHeatExchangeRatioHeating': None,
    'AirHeatExchangerEffectiveAirVolumeRatio': None,
    'AirHeatExchangerControl': '無',
    'AirHeatExchangerPowerConsumption': None,
    'Info': 'EHP1-1O',
    'energy_consumption_ratio': array([1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1.2]),
    'FanPowerConsumption_total': 0.05},
   {'Type': '全熱交ユニット',
    'Number': 1.0,
    'RatedCapacityCooling': None,
    'RatedCapacityHeating': None,
    'FanType': None,

In [80]:
inputdata["AirConditioningZone"]

{'1F_ロビー': {'isNatualVentilation': '無',
  'isSimultaneousSupply': '無',
  'AHU_cooling_insideLoad': 'FCU1-1',
  'AHU_cooling_outdoorLoad': 'FCU1-1',
  'AHU_heating_insideLoad': 'FCU1-1',
  'AHU_heating_outdoorLoad': 'FCU1-1',
  'Info': None,
  'buildingType': '事務所等',
  'roomType': 'ロビー',
  'zoneArea': 114.12,
  'ceilingHeight': 3.5,
  'outdoorAirVolume': 285.3},
 '1F_EVホール': {'isNatualVentilation': '無',
  'isSimultaneousSupply': '無',
  'AHU_cooling_insideLoad': 'FCU1-2',
  'AHU_cooling_outdoorLoad': 'FCU1-2',
  'AHU_heating_insideLoad': 'FCU1-2',
  'AHU_heating_outdoorLoad': 'FCU1-2',
  'Info': None,
  'buildingType': '事務所等',
  'roomType': '廊下',
  'zoneArea': 16.32,
  'ceilingHeight': 3.5,
  'outdoorAirVolume': 40.8},
 '1F_中央監視室・警備室': {'isNatualVentilation': '無',
  'isSimultaneousSupply': '無',
  'AHU_cooling_insideLoad': 'EHP1-1',
  'AHU_cooling_outdoorLoad': 'EHP1-1',
  'AHU_heating_insideLoad': 'EHP1-1',
  'AHU_heating_outdoorLoad': 'EHP1-1',
  'Info': None,
  'buildingType': '事務所等',


In [81]:
inputdata["AirConditioningZone"][room_zone_name]["AHU_cooling_insideLoad"]

'HU-72'

In [82]:
inputdata["AirConditioningZone"][room_zone_name]["AHU_cooling_outdoorLoad"]

'HU-72'

In [83]:
inputdata["AirConditioningZone"][room_zone_name]["outdoorAirVolume"]

2497.5

In [84]:
inputdata["HeatsourceSystem"]

{'AR1': {'冷房': {'StorageType': None,
   'StorageSize': None,
   'isStagingControl': '有',
   'Heatsource': [{'HeatsourceType': '吸収式冷凍機(一重二重併用形、都市ガス)',
     'Number': 1.0,
     'SupplyWaterTempSummer': 7.0,
     'SupplyWaterTempMiddle': 7.0,
     'SupplyWaterTempWinter': 7.0,
     'HeatsourceRatedCapacity': 703.0,
     'HeatsourceRatedPowerConsumption': 0,
     'HeatsourceRatedFuelConsumption': 639.0,
     'Heatsource_sub_RatedPowerConsumption': 6.6,
     'PrimaryPumpPowerConsumption': 7.5,
     'PrimaryPumpContolType': '無',
     'CoolingTowerCapacity': 1233.0,
     'CoolingTowerFanPowerConsumption': 7.4,
     'CoolingTowerPumpPowerConsumption': 15.0,
     'CoolingTowerContolType': '無',
     'Info': None,
     'HeatsourceRatedCapacity_total': 703.0,
     'HeatsourceRatedPowerConsumption_total': 0.0,
     'HeatsourceRatedFuelConsumption_total': 639.0,
     'Heatsource_sub_RatedPowerConsumption_total': 6.6,
     'PrimaryPumpPowerConsumption_total': 7.5,
     'CoolingTowerFanPowerConsumptio